(Using Julia to do text pre-processing)

In [18]:
using Pkg
pkg"activate ."

 Activating environment at `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Project.toml`


In [19]:
pkg"add PyCall" # to run Python code from Julia
pkg"add StatsBase" # to pick masking tokens

  Resolving package versions...
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Project.toml`
 [no changes]
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Project.toml`
 [no changes]
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Manifest.toml`
 [no changes]


In [20]:
using PyCall
py"""
import nlp
import numpy as np
from transformers import BertTokenizer

def extract_rows():
    dataset = nlp.load_dataset('glue', "sst2")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    results = []
    for sent, label in zip(dataset['train']['sentence'], dataset['train']['label']):
        results.append((tokenizer.tokenize(sent), label))
    return results
"""

In [21]:
@time rows = py"extract_rows"();

 13.812554 seconds (4.24 M allocations: 145.097 MiB, 0.91% gc time)


In [22]:
@show rows[1][1]
@show startswith.(rows[1][1], "##")

(rows[1])[1] = ["hide", "new", "secret", "##ions", "from", "the", "parental", "units"]
startswith.((rows[1])[1], "##") = Bool[0, 0, 0, 1, 0, 0, 0, 0]


8-element BitArray{1}:
 0
 0
 0
 1
 0
 0
 0
 0

In [23]:
sentences = [x[1] for x in rows]
function is_first_piece(arr::Array{String,1})
    return .!startswith.(arr, "##")
end
is_first_piece(sentences[1])

8-element BitArray{1}:
 1
 1
 1
 0
 1
 1
 1
 1

In [24]:
@time first_piece_masks = is_first_piece.(sentences)

  0.029666 seconds (202.56 k allocations: 11.958 MiB)


67349-element Array{BitArray{1},1}:
 [1, 1, 1, 0, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 0, 1, 0]
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
 [1, 1, 0, 0]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1]
 [1, 1, 1, 0, 0, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1]
 ⋮
 [1, 1, 1, 0, 1, 1, 0, 0]
 [1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 0]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1]
 [1, 1, 1]
 [1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1]
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [58]:
using StatsBase
function sample_mask_position(first_piece_mask, n=1)
    if sum(first_piece_mask) <= n
        return []
    end
    return sample(1:length(first_piece_mask), Weights(first_piece_mask), n, replace=false)
end

sample_mask_position (generic function with 2 methods)

In [26]:
masking_points = sample_mask_position.(first_piece_masks)

67349-element Array{Array{Int64,1},1}:
 [8]
 [1]
 [12]
 [2]
 [7]
 [4]
 [16]
 [2]
 [5]
 [9]
 [4]
 [19]
 [2]
 ⋮
 [3]
 [2]
 [2]
 [2]
 [10]
 [13]
 [2]
 [1]
 [5]
 [12]
 [2]
 [1]

In [59]:
@time begin
    buffer = Array{Tuple{Array{String,1},Int64},1}[rows]
    for round in 1:2
        augmented_rows = deepcopy(rows)
        masking_points = sample_mask_position.(first_piece_masks, round)
        for idx in 1:length(masking_points)
            for pos in masking_points[idx]
                @assert first_piece_masks[idx][pos] == 1
                augmented_rows[idx][1][pos] = "[MASK]"
                while pos + 1 <= length(first_piece_masks[idx]) && first_piece_masks[idx][pos + 1] == 0 
                    pos += 1
                    augmented_rows[idx][1][pos] = "[MASK]"
                end
            end
        end
        push!(buffer, augmented_rows)
    end
end

  0.519033 seconds (1.55 M allocations: 95.367 MiB, 4.44% gc time)


In [60]:
buffer[1][1:3]

3-element Array{Tuple{Array{String,1},Int64},1}:
 (["hide", "new", "secret", "##ions", "from", "the", "parental", "units"], 0)
 (["contains", "no", "wit", ",", "only", "labor", "##ed", "gag", "##s"], 0)
 (["that", "loves", "its", "characters", "and", "communicate", "##s", "something", "rather", "beautiful", "about", "human", "nature"], 1)

In [61]:
buffer[2][1:3]

3-element Array{Tuple{Array{String,1},Int64},1}:
 (["hide", "new", "secret", "##ions", "from", "the", "[MASK]", "units"], 0)
 (["contains", "no", "wit", ",", "[MASK]", "labor", "##ed", "gag", "##s"], 0)
 (["that", "loves", "[MASK]", "characters", "and", "communicate", "##s", "something", "rather", "beautiful", "about", "human", "nature"], 1)

In [62]:
buffer[3][1:3]

3-element Array{Tuple{Array{String,1},Int64},1}:
 (["hide", "[MASK]", "[MASK]", "[MASK]", "from", "the", "parental", "units"], 0)
 (["contains", "[MASK]", "wit", ",", "only", "labor", "##ed", "[MASK]", "[MASK]"], 0)
 (["that", "loves", "its", "characters", "and", "[MASK]", "[MASK]", "[MASK]", "rather", "beautiful", "about", "human", "nature"], 1)

In [63]:
all_rows = vcat(buffer...);

In [64]:
py"""
import numpy as np

def export(obj):
    with open("../cache/from_julia.npz", "wb") as f:
        np.save(f, obj)
"""

In [65]:
py"export"(all_rows)